In [4]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

%matplotlib widget
sns.set()
pd.options.display.float_format = '{:,.2f}'.format

FIGSIZE = (10, 10)
XRANGE = (0, 5)
YRANGE = (0, 2.5)

In [5]:
df = pd.read_excel('Closings.xlsx')
df.columns = ['ID', 'LiqRev', 'Inventory', 'Type', 'LYRev', 'HHI', 'Age']
df['LiqRevMM'] = df['LiqRev'] / 1000000
df['InventoryMM'] = df['Inventory'] / 1000000
df['LYRevMM'] = df['LYRev'] / 1000000

# Plot Liquidation Revenue vs. Inventory

In [6]:
fig, ax = plt.subplots(figsize=FIGSIZE)
sns.scatterplot(x='InventoryMM', y='LiqRevMM', data=df, ax=ax)
ax.set_xlabel('Inventory (MM)')
ax.set_ylabel('Liquidation Revenue (MM)')
plt.xlim(XRANGE)
plt.ylim(YRANGE)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Finding a Best Fit Line


In [12]:
fig, ax = plt.subplots(figsize=FIGSIZE)
sns.scatterplot(x='InventoryMM', y='LiqRevMM', data=df, ax=ax)
ax.set_xlabel('Inventory (MM)')
ax.set_ylabel('Liquidation Revenue (MM)')

testa = .1
testb = 0.5

ax.add_line(mlines.Line2D(XRANGE, 
                          [testa + testb*x for x in XRANGE],
                           color='darkred'))

plt.xlim(XRANGE)
plt.ylim(YRANGE)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Best Fit for Liquidation Revenue as a Function of Inventory

In [8]:
results = smf.ols('LiqRevMM ~ InventoryMM', data=df).fit()

fig, ax = plt.subplots(figsize=FIGSIZE)
sns.scatterplot(x='InventoryMM', y='LiqRevMM', data=df, ax=ax)
ax.set_xlabel('Inventory (MM)')
ax.set_ylabel('Liquidation Revenue (MM)')

ax.add_line(mlines.Line2D(XRANGE, 
                          [results.params[0] + results.params[1]*x
                           for x in XRANGE],
                           color='darkred'))

plt.xlim(XRANGE)
plt.ylim(YRANGE)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Liquidation Revenue as a Function of Inventory and Last-Year Revenue

In [13]:
results = smf.ols('LiqRevMM ~ InventoryMM + LYRevMM', data=df).fit()
results.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                 Results: Ordinary least squares
==================================================================
Model:              OLS              Adj. R-squared:     0.957    
Dependent Variable: LiqRevMM         AIC:                -566.4478
Date:               2020-03-25 09:26 BIC:                -556.5982
No. Observations:   197              Log-Likelihood:     286.22   
Df Model:           2                F-statistic:        2206.    
Df Residuals:       194              Prob (F-statistic): 3.81e-134
R-squared:          0.958            Scale:              0.0032524
--------------------------------------------------------------------
              Coef.    Std.Err.      t      P>|t|    [0.025   0.975]
--------------------------------------------------------------------
Intercept     0.1376     0.0159    8.6385   0.0000   0.1062   0.1690
InventoryMM   0.4403     0.0134   32.7400   0.0000   0.4138   0.4668
LYRevMM       0.1693     0.0379    4.4692   0.0000   0.0946   0.2440
------------------------------------------------------------------
Omnibus:              9.656        Durbin-Watson:           1.921 
Prob(Omnibus):        0.008        Jarque-Bera (JB):        18.811
Skew:                 0.136        Prob(JB):                0.000 
Kurtosis:             4.489        Condition No.:           23    
==================================================================

"""

In [14]:
fig = plt.figure(figsize=FIGSIZE)
ax = fig.add_subplot(projection='3d')

mesh_divs = 5

surfaces = np.meshgrid(np.arange(0, df['InventoryMM'].max() + 1,
                                 df['InventoryMM'].max() / mesh_divs),
                       np.arange(0, df['LYRevMM'].max() + 1,
                                 df['LYRevMM'].max() / mesh_divs))

df['Prediction'] = results.predict()
dfhigh = df[df['Prediction'] > df['LiqRevMM']]
dflow = df[df['Prediction'] <= df['LiqRevMM']]

pred = results.predict(exog=pd.DataFrame({'InventoryMM': surfaces[0].ravel(),
                                          'LYRevMM': surfaces[1].ravel()}))

ax.plot_surface(surfaces[0], surfaces[1],
                pred.values.reshape(surfaces[0].shape),
                rstride=1, cstride=1,
                color='blue', alpha=0.2)

ax.scatter(xs=dfhigh['InventoryMM'], ys=dfhigh['LYRevMM'],
           zs=dfhigh['LiqRevMM'], color='Green')

ax.scatter(xs=dflow['InventoryMM'], ys=dflow['LYRevMM'],
           zs=dflow['LiqRevMM'], color='Red')

ax.set_xlabel('Inventory (MM)')
ax.set_ylabel('LYRev (MM)')
ax.set_zlabel('LiqRev (MM)')

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …